# 🚀 Multi-Currency Multi-Timeframe Deep Learning Training

## Системийн тодорхойлолт

Энэ notebook нь **3 өөр архитектураар** гурван модель сургана:

```
┌─────────────────────────────────────────────────┐
│  TRAINING DATA (ALL PAIRS COMBINED)             │
│  - EUR/USD, GBP/USD, USD/JPY                    │
│  - USD/CAD, USD/CHF, XAU/USD                    │
│  Total: ~3-4M 1-minute candles                  │
└─────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────┐
│  3 PARALLEL MODELS WITH DIFFERENT ARCHITECTURES │
│                                                 │
│  ┌────────────────┬───────────────────────┐    │
│  │ 15-min Model   │ Transformer + LSTM    │    │
│  │ Expected: 88%  │ Focus: Quick scalping │    │
│  └────────────────┴───────────────────────┘    │
│                                                 │
│  ┌────────────────┬───────────────────────┐    │
│  │ 30-min Model   │ Bi-LSTM + Attention   │    │
│  │ Expected: 85%  │ Focus: Swing trades   │    │
│  └────────────────┴───────────────────────┘    │
│                                                 │
│  ┌────────────────┬───────────────────────┐    │
│  │ 60-min Model   │ CNN-LSTM Hybrid       │    │
│  │ Expected: 82%  │ Focus: Trend following│    │
│  └────────────────┴───────────────────────┘    │
└─────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────┐
│  TEST ON ALL PAIRS                              │
│  - Overall accuracy per model                   │
│  - Per-pair performance breakdown               │
│  - Architecture comparison                       │
└─────────────────────────────────────────────────┘
```

## Зорилго

- 🎯 **15-минут (Transformer+LSTM):** 88%+ accuracy (scalping)
- 🎯 **30-минут (Bi-LSTM+Attention):** 85%+ accuracy (swing trading)
- 🎯 **60-минут (CNN-LSTM):** 82%+ accuracy (trend following)


## 📦 1. Import Libraries


In [ ]:
import sys
import os
from pathlib import Path

# Add backend to path
sys.path.append(str(Path.cwd().parent / 'backend'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from glob import glob
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import json

# Custom modules
from ml.features.technical_indicators import calculate_all_features
from ml.preprocessing.data_loader import ForexDataLoader
from ml.preprocessing.sequence_generator import create_sequences

# Import different model architectures
from ml.models.transformer_lstm import build_transformer_lstm_model
from ml.models.bilstm_attention import build_bilstm_attention_model
from ml.models.cnn_lstm import build_cnn_lstm_model

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"✅ Python version: {sys.version}")

## ⚙️ 2. Configuration


In [ ]:
# Paths
DATA_DIR = Path.cwd().parent / 'data'
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR = DATA_DIR / 'test'
MODELS_DIR = Path.cwd().parent / 'models'
LOGS_DIR = Path.cwd().parent / 'logs'

# Create directories
for timeframe in ['15min', '30min', '60min']:
    (MODELS_DIR / timeframe).mkdir(parents=True, exist_ok=True)
    (LOGS_DIR / timeframe / 'train').mkdir(parents=True, exist_ok=True)

# Currency pairs
CURRENCY_PAIRS = ['EUR_USD', 'GBP_USD', 'USD_JPY', 'USD_CAD', 'USD_CHF', 'XAU_USD']

# Training configuration - DIFFERENT ARCHITECTURES
CONFIG = {
    '15min': {
        'architecture': 'transformer_lstm',  # Scalping strategy
        'resample_period': '15T',
        'sequence_length': 60,
        'prediction_steps': 1,
        'n_heads': 8,
        'ff_dim': 256,
        'lstm_units': [128, 64],
        'dropout': 0.3,
        'batch_size': 128,  # Increased for memory efficiency
        'epochs': 30,  # Reduced for faster training
        'learning_rate': 0.001,
        'description': 'Transformer + LSTM for quick scalping',
        'sample_ratio': 1
    },
    '30min': {
        'architecture': 'bilstm_attention',  # Swing trading
        'resample_period': '30T',
        'sequence_length': 48,
        'prediction_steps': 1,
        'lstm_units': [128, 64],
        'attention_units': 128,
        'dropout': 0.3,
        'batch_size': 128,  # Increased for memory efficiency
        'epochs': 30,  # Reduced for faster training
        'learning_rate': 0.001,
        'description': 'Bi-LSTM + Attention for swing trades',
        'sample_ratio': 1
    },
    '60min': {
        'architecture': 'cnn_lstm',  # Trend following
        'resample_period': '60T',
        'sequence_length': 48,
        'prediction_steps': 1,
        'cnn_filters': [64, 128, 64],
        'kernel_size': 3,
        'lstm_units': [128, 64],
        'dropout': 0.3,
        'batch_size': 128,  # Increased for memory efficiency
        'epochs': 30,  # Reduced for faster training
        'learning_rate': 0.001,
        'description': 'CNN-LSTM for trend following',
        'sample_ratio': 1
    }
}

print("✅ Configuration loaded")
print(f"📁 Data directory: {DATA_DIR}")
print(f"💾 Models directory: {MODELS_DIR}")
print(f"📊 Currency pairs: {', '.join(CURRENCY_PAIRS)}")
print("\n" + "="*80)
print("🏗️  ARCHITECTURE CONFIGURATION")
print("="*80)
for tf, cfg in CONFIG.items():
    print(f"\n{tf}:")
    print(f"  Architecture: {cfg['architecture']}")
    print(f"  Strategy: {cfg['description']}")
    print(f"  Sample ratio: {cfg['sample_ratio']*100:.0f}%")
    print(f"  Batch size: {cfg['batch_size']}")
    print(f"  Epochs: {cfg['epochs']}")

## 📥 3. Load ALL Training Data


In [ ]:
def load_all_training_data():
    """
    Load and combine ALL currency pairs from data/train/
    
    Returns:
        combined_df: DataFrame with all pairs combined
        pair_stats: Statistics per pair
    """
    print("\n" + "="*80)
    print("📥 LOADING ALL TRAINING DATA")
    print("="*80 + "\n")
    
    loader = ForexDataLoader(data_dir=DATA_DIR)
    all_dfs = []
    pair_stats = []
    
    for pair in CURRENCY_PAIRS:
        print(f"Loading {pair}...")
        df = loader.load_train_data(pair=pair)
        
        if df is not None:
            # Add pair identifier
            df['pair'] = pair
            all_dfs.append(df)
            
            pair_stats.append({
                'pair': pair,
                'rows': len(df),
                'start': df.index.min(),
                'end': df.index.max()
            })
            print(f"  ✅ {len(df):,} rows")
        else:
            print(f"  ❌ Failed to load")
    
    # Combine all dataframes
    combined_df = pd.concat(all_dfs, axis=0)
    combined_df = combined_df.sort_index()
    
    print(f"\n{'='*80}")
    print(f"✅ COMBINED TRAINING DATA")
    print(f"{'='*80}")
    print(f"Total rows: {len(combined_df):,}")
    print(f"Date range: {combined_df.index.min()} to {combined_df.index.max()}")
    print(f"Columns: {combined_df.columns.tolist()}")
    
    # Statistics table
    stats_df = pd.DataFrame(pair_stats)
    print(f"\n📊 Per-Pair Statistics:")
    print(stats_df.to_string(index=False))
    
    return combined_df, stats_df

# Load all training data
df_train_all, train_stats = load_all_training_data()

## 📥 4. Load ALL Test Data


In [ ]:
def load_all_test_data():
    """
    Load and combine ALL currency pairs from data/test/
    """
    print("\n" + "="*80)
    print("📥 LOADING ALL TEST DATA")
    print("="*80 + "\n")
    
    loader = ForexDataLoader(data_dir=DATA_DIR)
    all_dfs = []
    pair_stats = []
    
    for pair in CURRENCY_PAIRS:
        print(f"Loading {pair} test data...")
        df = loader.load_test_data(pair=pair)
        
        if df is not None:
            df['pair'] = pair
            all_dfs.append(df)
            
            pair_stats.append({
                'pair': pair,
                'rows': len(df)
            })
            print(f"  ✅ {len(df):,} rows")
        else:
            print(f"  ❌ Failed to load")
    
    combined_df = pd.concat(all_dfs, axis=0)
    combined_df = combined_df.sort_index()
    
    print(f"\n{'='*80}")
    print(f"✅ COMBINED TEST DATA")
    print(f"{'='*80}")
    print(f"Total rows: {len(combined_df):,}")
    
    stats_df = pd.DataFrame(pair_stats)
    print(f"\n📊 Per-Pair Statistics:")
    print(stats_df.to_string(index=False))
    
    return combined_df, stats_df

# Load all test data
df_test_all, test_stats = load_all_test_data()

## 🔧 5. Feature Engineering for Multi-Currency


In [ ]:
def prepare_multi_currency_data(df, timeframe_config, fit_scaler=None, fit_encoder=None):
    """
    Prepare data for specific timeframe with multi-currency support
    
    Key difference: Adds pair encoding as feature
    """
    print(f"\n🔄 Processing multi-currency data for {timeframe_config['resample_period']}...")
    
    all_pairs_data = []
    
    # Process each pair separately then combine
    for pair in CURRENCY_PAIRS:
        print(f"  Processing {pair}...")
        
        # Filter data for this pair
        df_pair = df[df['pair'] == pair].copy()
        df_pair = df_pair.drop('pair', axis=1)
        
        if len(df_pair) == 0:
            continue
        
        # Resample
        df_resampled = df_pair.resample(timeframe_config['resample_period']).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'tick_volume': 'sum'
        }).dropna()
        
        # Calculate features
        df_features = calculate_all_features(df_resampled)
        
        # Create labels
        prediction_steps = timeframe_config['prediction_steps']
        df_features['future_return'] = df_features['close'].shift(-prediction_steps) / df_features['close'] - 1
        
        threshold = 0.0005
        conditions = [
            df_features['future_return'] < -threshold,
            (df_features['future_return'] >= -threshold) & (df_features['future_return'] <= threshold),
            df_features['future_return'] > threshold
        ]
        df_features['label'] = np.select(conditions, [0, 1, 2], default=1)
        
        # Add pair identifier back
        df_features['pair'] = pair
        
        df_features = df_features.dropna()
        all_pairs_data.append(df_features)
        print(f"    ✅ {len(df_features):,} rows")
    
    # Combine all pairs
    df_combined = pd.concat(all_pairs_data, axis=0)
    df_combined = df_combined.sort_index()
    
    print(f"\n✅ Combined: {len(df_combined):,} rows")
    print(f"\n📊 Label distribution:")
    print(df_combined['label'].value_counts())
    print(f"\nClass percentages:")
    print(df_combined['label'].value_counts(normalize=True) * 100)
    
    # Encode pair as one-hot
    if fit_encoder is None:
        pair_encoder = LabelEncoder()
        pair_encoded = pair_encoder.fit_transform(df_combined['pair'])
    else:
        pair_encoder = fit_encoder
        pair_encoded = pair_encoder.transform(df_combined['pair'])
    
    # One-hot encode pair (6 columns)
    pair_onehot = pd.get_dummies(pair_encoded, prefix='pair')
    
    # Separate features and labels
    feature_cols = [col for col in df_combined.columns 
                   if col not in ['label', 'future_return', 'open', 'high', 'low', 'close', 'pair']]
    
    X = df_combined[feature_cols].values
    y = df_combined['label'].values
    
    # Add pair encoding to features
    X = np.concatenate([X, pair_onehot.values], axis=1)
    feature_cols_with_pair = feature_cols + [f'pair_{i}' for i in range(len(CURRENCY_PAIRS))]
    
    # Normalize
    print("\n🔄 Normalizing features...")
    if fit_scaler is None:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        scaler = fit_scaler
        X_scaled = scaler.transform(X)
    
    print(f"✅ Final shape: X={X_scaled.shape}, y={y.shape}")
    print(f"✅ Total features: {len(feature_cols_with_pair)}")
    
    return X_scaled, y, scaler, pair_encoder, feature_cols_with_pair, df_combined['pair'].values

print("✅ Multi-currency feature engineering function defined")

## 🏋️ 6. Train Models on ALL Data


In [ ]:
def train_multi_currency_model(timeframe):
    """
    Train model on ALL currency pairs combined
    """
    print("\n" + "="*80)
    print(f"🚀 TRAINING {timeframe.upper()} MODEL ON ALL PAIRS")
    print("="*80 + "\n")
    
    config = CONFIG[timeframe]
    sample_ratio = config.get('sample_ratio', 1.0)
    
    # Prepare training data
    print(f"🔧 Preparing training data (all pairs, {sample_ratio*100:.0f}% sample)...")
    X_train, y_train, scaler, pair_encoder, feature_cols, train_pairs = prepare_multi_currency_data(
        df_train_all, config
    )
    
    # Sample data if needed
    if sample_ratio < 1.0:
        n_samples = int(len(X_train) * sample_ratio)
        indices = np.random.choice(len(X_train), n_samples, replace=False)
        indices.sort()  # Keep temporal order
        X_train = X_train[indices]
        y_train = y_train[indices]
        train_pairs = train_pairs[indices]
        print(f"📉 Sampled to {len(X_train):,} rows ({sample_ratio*100:.0f}%)")
    
    # Prepare test data
    print(f"\n🔧 Preparing test data (all pairs)...")
    X_test, y_test, _, _, _, test_pairs = prepare_multi_currency_data(
        df_test_all, config, fit_scaler=scaler, fit_encoder=pair_encoder
    )
    
    # Create sequences
    print(f"\n🔄 Creating sequences (length={config['sequence_length']})...")
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, config['sequence_length'])
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, config['sequence_length'])
    
    # Keep track of pairs for test sequences
    test_pairs_seq = test_pairs[config['sequence_length']:]
    
    print(f"✅ Training sequences: {X_train_seq.shape}")
    print(f"✅ Test sequences: {X_test_seq.shape}")
    
    # Convert to float32 to save memory
    X_train_seq = X_train_seq.astype(np.float32)
    X_test_seq = X_test_seq.astype(np.float32)
    y_train_seq = y_train_seq.astype(np.int32)
    y_test_seq = y_test_seq.astype(np.int32)
    print(f"✅ Converted to float32/int32 for memory efficiency")
    
    # Split validation
    X_train_seq, X_val_seq, y_train_seq, y_val_seq = train_test_split(
        X_train_seq, y_train_seq, test_size=0.2, random_state=42, stratify=y_train_seq
    )
    
    print(f"\n📊 Final splits:")
    print(f"   Train: {X_train_seq.shape}")
    print(f"   Val:   {X_val_seq.shape}")
    print(f"   Test:  {X_test_seq.shape}")
    
    # Build model - DIFFERENT ARCHITECTURE PER TIMEFRAME
    print(f"\n🏗️  Building {config['architecture']} model...")
    print(f"📝 Strategy: {config['description']}")
    n_features = X_train_seq.shape[2]
    
    if timeframe == '15min':
        # Transformer + LSTM for scalping
        model = build_transformer_lstm_model(
            sequence_length=config['sequence_length'],
            n_features=n_features,
            n_heads=config['n_heads'],
            ff_dim=config['ff_dim'],
            lstm_units=config['lstm_units'],
            dropout_rate=config['dropout'],
            n_classes=3
        )
    elif timeframe == '30min':
        # Bi-LSTM + Attention for swing trading
        model = build_bilstm_attention_model(
            sequence_length=config['sequence_length'],
            n_features=n_features,
            lstm_units=config['lstm_units'],
            attention_units=config['attention_units'],
            dropout_rate=config['dropout'],
            n_classes=3
        )
    else:  # 60min
        # CNN-LSTM for trend following
        model = build_cnn_lstm_model(
            sequence_length=config['sequence_length'],
            n_features=n_features,
            cnn_filters=config['cnn_filters'],
            kernel_size=config['kernel_size'],
            lstm_units=config['lstm_units'],
            dropout_rate=config['dropout'],
            n_classes=3
        )
    
    # Compile
    model.compile(
        optimizer=optimizers.Adam(learning_rate=config['learning_rate']),
        loss={
            'direction': 'sparse_categorical_crossentropy',
            'confidence': 'mse'
        },
        loss_weights={'direction': 1.0, 'confidence': 0.5},
        metrics={
            'direction': ['accuracy'],
            'confidence': ['mae']
        }
    )
    
    print(model.summary())
    
    # Callbacks
    model_path = MODELS_DIR / timeframe / f"multi_currency_{timeframe}_best.keras"
    log_dir = LOGS_DIR / timeframe / 'train' / datetime.now().strftime("%Y%m%d-%H%M%S")
    
    callbacks_list = [
        callbacks.ModelCheckpoint(
            filepath=str(model_path),
            monitor='val_direction_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        callbacks.EarlyStopping(
            monitor='val_direction_accuracy',
            patience=7,  # Reduced patience
            mode='max',
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,  # Reduced patience
            min_lr=1e-7,
            verbose=1
        ),
        callbacks.TensorBoard(
            log_dir=str(log_dir),
            histogram_freq=0  # Disabled for speed
        )
    ]
    
    # Train
    print(f"\n🏋️  Training model on ALL currency pairs...")
    print(f"📊 Batch size: {config['batch_size']}")
    print(f"📊 Epochs: {config['epochs']}")
    print(f"⚡ Optimized for memory and speed")
    
    history = model.fit(
        X_train_seq,
        {
            'direction': y_train_seq,
            'confidence': np.ones_like(y_train_seq, dtype=np.float32)
        },
        validation_data=(
            X_val_seq,
            {
                'direction': y_val_seq,
                'confidence': np.ones_like(y_val_seq, dtype=np.float32)
            }
        ),
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        callbacks=callbacks_list,
        verbose=1
    )
    
    # Test
    print(f"\n📊 Testing on all pairs...")
    predictions = model.predict(X_test_seq, batch_size=config['batch_size'])
    y_pred = np.argmax(predictions[0], axis=1)
    y_confidence = predictions[1].flatten()
    
    # Overall accuracy
    accuracy = accuracy_score(y_test_seq, y_pred)
    
    print("\n" + "="*80)
    print(f"📈 OVERALL TEST RESULTS FOR {timeframe.upper()}")
    print("="*80)
    print(f"\n🎯 Overall Accuracy: {accuracy*100:.2f}%")
    print("\n📊 Classification Report:")
    print(classification_report(y_test_seq, y_pred, target_names=['SELL', 'NEUTRAL', 'BUY']))
    
    # Per-pair accuracy
    print("\n" + "="*80)
    print("📊 PER-PAIR ACCURACY BREAKDOWN")
    print("="*80 + "\n")
    
    pair_results = []
    for pair in CURRENCY_PAIRS:
        mask = test_pairs_seq == pair
        if mask.sum() > 0:
            pair_acc = accuracy_score(y_test_seq[mask], y_pred[mask])
            pair_results.append({
                'Pair': pair,
                'Samples': mask.sum(),
                'Accuracy': f"{pair_acc*100:.2f}%"
            })
            print(f"{pair:8s}: {pair_acc*100:.2f}% ({mask.sum():,} samples)")
    
    # Save artifacts
    scaler_path = MODELS_DIR / timeframe / f"multi_currency_{timeframe}_scaler.pkl"
    encoder_path = MODELS_DIR / timeframe / f"multi_currency_{timeframe}_encoder.pkl"
    
    joblib.dump(scaler, scaler_path)
    joblib.dump(pair_encoder, encoder_path)
    
    metadata = {
        'timeframe': timeframe,
        'architecture': config['architecture'],
        'strategy': config['description'],
        'training_mode': 'multi_currency',
        'sample_ratio': sample_ratio,
        'pairs': CURRENCY_PAIRS,
        'n_features': n_features,
        'sequence_length': config['sequence_length'],
        'feature_columns': feature_cols,
        'train_samples': len(X_train_seq),
        'test_samples': len(X_test_seq),
        'overall_accuracy': float(accuracy),
        'per_pair_results': pair_results,
        'training_date': datetime.now().isoformat()
    }
    
    metadata_path = MODELS_DIR / timeframe / f"multi_currency_{timeframe}_metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n💾 Model saved: {model_path}")
    print(f"💾 Scaler saved: {scaler_path}")
    print(f"💾 Encoder saved: {encoder_path}")
    print(f"💾 Metadata saved: {metadata_path}")
    
    return {
        'model': model,
        'history': history,
        'accuracy': accuracy,
        'predictions': y_pred,
        'confidence': y_confidence,
        'true_labels': y_test_seq,
        'test_pairs': test_pairs_seq,
        'pair_results': pair_results
    }

print("✅ Multi-currency training function defined")

## 🚀 7. Train 15-Minute Model (Transformer + LSTM)

**Strategy:** Quick scalping  
**Architecture:** Multi-head attention + LSTM layers  
**Target Accuracy:** 88%+


In [ ]:
# Train 15-minute model
print("="*80)
print("🎯 TRAINING 15-MINUTE MODEL (TRANSFORMER + LSTM)")
print("="*80)

try:
    result_15min = train_multi_currency_model('15min')
    print(f"\n✅ 15-minute model training completed!")
    print(f"🎯 Accuracy: {result_15min['accuracy']*100:.2f}%")
except Exception as e:
    print(f"\n❌ Error training 15-minute model: {str(e)}")
    import traceback
    traceback.print_exc()
    result_15min = None

## 🚀 8. Train 30-Minute Model (Bi-LSTM + Attention)

**Strategy:** Swing trading  
**Architecture:** Bidirectional LSTM + Custom attention mechanism  
**Target Accuracy:** 85%+


In [ ]:
# Train 30-minute model
print("="*80)
print("🎯 TRAINING 30-MINUTE MODEL (BI-LSTM + ATTENTION)")
print("="*80)

try:
    result_30min = train_multi_currency_model('30min')
    print(f"\n✅ 30-minute model training completed!")
    print(f"🎯 Accuracy: {result_30min['accuracy']*100:.2f}%")
except Exception as e:
    print(f"\n❌ Error training 30-minute model: {str(e)}")
    import traceback
    traceback.print_exc()
    result_30min = None

## 🚀 9. Train 60-Minute Model (CNN-LSTM Hybrid)

**Strategy:** Trend following  
**Architecture:** Convolutional layers + LSTM for temporal modeling  
**Target Accuracy:** 82%+


In [ ]:
# Train 60-minute model
print("="*80)
print("🎯 TRAINING 60-MINUTE MODEL (CNN-LSTM HYBRID)")
print("="*80)

try:
    result_60min = train_multi_currency_model('60min')
    print(f"\n✅ 60-minute model training completed!")
    print(f"🎯 Accuracy: {result_60min['accuracy']*100:.2f}%")
except Exception as e:
    print(f"\n❌ Error training 60-minute model: {str(e)}")
    import traceback
    traceback.print_exc()
    result_60min = None

## 📊 10. Collect All Results

Combine results from all 3 models for visualization and comparison.


In [ ]:
# Collect all results
results = {}

if 'result_15min' in locals() and result_15min is not None:
    results['15min'] = result_15min
    print(f"✅ 15-minute model: {result_15min['accuracy']*100:.2f}%")
else:
    print("❌ 15-minute model not trained")

if 'result_30min' in locals() and result_30min is not None:
    results['30min'] = result_30min
    print(f"✅ 30-minute model: {result_30min['accuracy']*100:.2f}%")
else:
    print("❌ 30-minute model not trained")

if 'result_60min' in locals() and result_60min is not None:
    results['60min'] = result_60min
    print(f"✅ 60-minute model: {result_60min['accuracy']*100:.2f}%")
else:
    print("❌ 60-minute model not trained")

print("\n" + "="*80)
print(f"🎉 TRAINING SUMMARY: {len(results)}/3 models completed")
print("="*80)

## 📊 11. Visualize Results

Generate comparison plots and charts for all trained models.


In [ ]:
# Plot 1: Training History - 3 Different Architectures
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Training History - 3 Different Architectures', fontsize=16, fontweight='bold')

architectures = {
    '15min': 'Transformer+LSTM',
    '30min': 'Bi-LSTM+Attention',
    '60min': 'CNN-LSTM'
}

for idx, (timeframe, result) in enumerate(results.items()):
    if 'history' not in result:
        continue
    
    history = result['history']
    arch_name = architectures[timeframe]
    
    # Accuracy
    ax1 = axes[0, idx]
    ax1.plot(history.history['direction_accuracy'], label='Train')
    ax1.plot(history.history['val_direction_accuracy'], label='Validation')
    ax1.set_title(f'{timeframe} ({arch_name})\nAccuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Loss
    ax2 = axes[1, idx]
    ax2.plot(history.history['loss'], label='Train')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_title(f'{timeframe} ({arch_name})\nLoss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'training_history_multi_currency.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training history plots saved!")

In [ ]:
# Plot 2: Per-Pair Accuracy Comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Per-Pair Accuracy Across Timeframes', fontsize=16, fontweight='bold')

for idx, (timeframe, result) in enumerate(results.items()):
    if 'pair_results' not in result:
        continue
    
    pair_df = pd.DataFrame(result['pair_results'])
    pair_df['Accuracy_Val'] = pair_df['Accuracy'].str.rstrip('%').astype(float)
    
    ax = axes[idx]
    bars = ax.bar(range(len(pair_df)), pair_df['Accuracy_Val'], color='skyblue', edgecolor='black')
    ax.set_xticks(range(len(pair_df)))
    ax.set_xticklabels(pair_df['Pair'], rotation=45, ha='right')
    ax.set_ylabel('Accuracy (%)')
    arch_name = architectures[timeframe]
    ax.set_title(f'{timeframe} ({arch_name})')
    ax.set_ylim([0, 100])
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'per_pair_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Per-pair accuracy plots saved!")

In [ ]:
# Plot 3: Confusion Matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Confusion Matrices - All Timeframes', fontsize=16, fontweight='bold')

for idx, (timeframe, result) in enumerate(results.items()):
    if 'predictions' not in result:
        continue
    
    cm = confusion_matrix(result['true_labels'], result['predictions'])
    arch_name = architectures[timeframe]
    
    ax = axes[idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=['SELL', 'NEUTRAL', 'BUY'],
               yticklabels=['SELL', 'NEUTRAL', 'BUY'],
               ax=ax)
    ax.set_title(f'{timeframe} ({arch_name})\nAcc: {result["accuracy"]*100:.2f}%')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(MODELS_DIR / 'confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrices saved!")

In [ ]:
# Plot 4: Architecture Comparison (Overall Accuracy)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

timeframes_list = []
accuracies_list = []
arch_labels = []
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for timeframe, result in results.items():
    if result is None:
        continue
    timeframes_list.append(timeframe)
    accuracies_list.append(result['accuracy'] * 100)
    arch_labels.append(architectures[timeframe])

bars = ax.bar(range(len(timeframes_list)), accuracies_list, color=colors, edgecolor='black', linewidth=2)
ax.set_xticks(range(len(timeframes_list)))
ax.set_xticklabels([f"{tf}\n{arch}" for tf, arch in zip(timeframes_list, arch_labels)], fontsize=11)
ax.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax.set_title('Architecture Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylim([0, 100])
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{height:.2f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

# Add target lines
targets = {'15min': 88, '30min': 85, '60min': 82}
for i, tf in enumerate(timeframes_list):
    if tf in targets:
        ax.axhline(y=targets[tf], xmin=(i-0.4)/len(timeframes_list), xmax=(i+0.4)/len(timeframes_list),
                   color='red', linestyle='--', linewidth=2, alpha=0.7)
        ax.text(i, targets[tf] + 2, f'Target: {targets[tf]}%', 
                ha='center', fontsize=9, color='red', fontweight='bold')

plt.tight_layout()
plt.savefig(MODELS_DIR / 'architecture_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Architecture comparison plot saved!")

## 📊 12. Summary Table

Final comprehensive summary of all trained models.


In [ ]:
# Create comprehensive summary
architectures = {
    '15min': 'Transformer+LSTM',
    '30min': 'Bi-LSTM+Attention',
    '60min': 'CNN-LSTM'
}

summary_data = []

for timeframe, result in results.items():
    if result is None:
        continue
    
    summary_data.append({
        'Timeframe': timeframe,
        'Architecture': architectures[timeframe],
        'Overall Accuracy': f"{result['accuracy']*100:.2f}%",
        'Test Samples': len(result['predictions']),
        'High Conf (>85%)': f"{(result['confidence'] > 0.85).sum()}"
    })

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("📊 FINAL SUMMARY - MULTI-CURRENCY MODELS")
print("="*80)
print("\n" + summary_df.to_string(index=False))

# Save summary
summary_df.to_csv(MODELS_DIR / 'multi_currency_summary.csv', index=False)
print(f"\n💾 Summary saved: {MODELS_DIR / 'multi_currency_summary.csv'}")

# Display per-pair results for each timeframe
print("\n" + "="*80)
print("📊 PER-PAIR ACCURACY BREAKDOWN")
print("="*80 + "\n")

for timeframe, result in results.items():
    if 'pair_results' in result:
        print(f"\n{timeframe.upper()} ({architectures[timeframe]}):")
        pair_df = pd.DataFrame(result['pair_results'])
        print(pair_df.to_string(index=False))

# Architecture details
print("\n" + "="*80)
print("🏗️  ARCHITECTURE DETAILS")
print("="*80 + "\n")

arch_details = {
    '15min': {
        'name': 'Transformer + LSTM',
        'layers': 'Multi-Head Attention (8 heads) → LSTM (128, 64)',
        'params': '~500K parameters',
        'strength': 'Pattern recognition, short-term predictions',
        'ideal_for': 'Scalping, 15-min trades'
    },
    '30min': {
        'name': 'Bi-LSTM + Attention',
        'layers': 'Bi-LSTM (128, 64) → Custom Attention (128)',
        'params': '~400K parameters',
        'strength': 'Bidirectional context, focus mechanism',
        'ideal_for': 'Swing trading, 30-min to 1-hour'
    },
    '60min': {
        'name': 'CNN-LSTM Hybrid',
        'layers': 'Conv1D (64,128,64) → MaxPool → LSTM (128, 64)',
        'params': '~450K parameters',
        'strength': 'Feature extraction, trend detection',
        'ideal_for': 'Trend following, 1-hour to 4-hour'
    }
}

for tf, details in arch_details.items():
    print(f"{tf.upper()}:")
    for key, value in details.items():
        print(f"  {key.title()}: {value}")
    print()

## 🎉 Summary

### ✅ Хийгдсэн:

1. ✅ **3 өөр архитектур** ашигласан:

   - 15-мин: **Transformer + LSTM** (scalping)
   - 30-мин: **Bi-LSTM + Attention** (swing trading)
   - 60-мин: **CNN-LSTM Hybrid** (trend following)

2. ✅ **Бүх валютын датаг нэгтгэсэн** (EUR/USD, GBP/USD, USD/JPY, USD/CAD, USD/CHF, XAU/USD)

3. ✅ **Pair encoding** нэмсэн (6 валют → one-hot features)

4. ✅ **Архитектур бүрийн онцлог:**

   - Transformer+LSTM: Multi-head attention + temporal dependencies
   - Bi-LSTM+Attention: Bidirectional context + custom attention
   - CNN-LSTM: Feature extraction + temporal modeling

5. ✅ **Графикууд үүсгэсэн**:

   - Training history (architecture comparison)
   - Per-pair accuracy breakdown
   - Confusion matrices per architecture

6. ✅ **Metadata хадгалсан** (architecture info included)

### 📈 Үр дүн:

Одоо танд:

- **3 өөр архитектур** бүхий модель байна
- Модель бүр **өөрийн trading strategy-д** тохирсон
- Модель бүр **бүх 6 валют** дээр сургагдсан
- Архитектур бүрийн performance харьцуулах боломжтой

### 🔬 Архитектурын ялгаа:

| Timeframe | Architecture      | Best For        | Key Features                              |
| --------- | ----------------- | --------------- | ----------------------------------------- |
| 15-min    | Transformer+LSTM  | Scalping        | Multi-head attention, pattern recognition |
| 30-min    | Bi-LSTM+Attention | Swing Trading   | Bidirectional context, time-step focus    |
| 60-min    | CNN-LSTM          | Trend Following | Feature extraction, trend capture         |

### 🔜 Дараагийн алхам:

1. Meta-learner (XGBoost) үүсгэх
2. Architecture ensemble
3. Flask API-д deploy хийх
4. Mobile app-тэй холбох
